# Synthesis & Enhanced Visualizations

**Purpose:** Synthesize findings from Notebooks 1-6, create polished visualizations, and optionally test scenarios.

**Notebook 6 Final Results:**
- **Net migration correlation:** r = 0.489 ± 0.016
- **Optimal parameters:** Economic (60%) + Network (40%) + Distance (10%)
- **Top destination accuracy:** 7/10 matches
- **Key finding:** Network effects rescue economic-only model (+0.22 improvement)

**This Notebook:**
1. Compare Graph Analysis (NB1-4) vs ABM (NB5-6) approaches
2. Create polished visualizations for presentation
3. (Optional) Test 1-2 simple scenarios

**Note:** Detailed report content will be prepared separately in Word document.

## Section 1: Setup & Load Results

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import defaultdict

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 150  # High quality for report

# Load data
df = pd.read_csv('../data/bilat_mig_40_countries.csv')
G = nx.DiGraph()
for _, row in df.iterrows():
    if row['da_pb_closed'] > 0:
        G.add_edge(row['orig'], row['dest'], weight=row['da_pb_closed'])

print(f"Network loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Section 2: Graph Analysis vs ABM Comparison

**What each approach revealed:**

### Comparison Table

| Aspect | Graph Analysis (NB1-4) | Agent-Based Model (NB5-6) |
|--------|------------------------|---------------------------|
| **Question** | *What* patterns exist? | *Why* do patterns emerge? |
| **Nature** | Static snapshot (2015 data) | Dynamic simulation (mechanisms) |
| **Key Findings** | • Extreme concentration (top 20 = 80%)<br>• Power-law (α=-1.81)<br>• Hub-spoke topology<br>• Myanmar high betweenness | • Economic pull (60% weight)<br>• Network effects crucial (40%)<br>• Distance as revealed preference<br>• Synergy: Network rescues Economics |
| **Metrics** | Degree, betweenness, clustering, power-law exponent | Correlation (r=0.49), destination accuracy (7/10), Gini coefficient |
| **Strengths** | • Fast computation<br>• Clear empirical patterns<br>• No assumptions needed | • Explains *mechanisms*<br>• Testable scenarios<br>• Quantifies contributions |
| **Limitations** | • Descriptive only<br>• No causality<br>• Can't test "what-if" | • Simplified agents<br>• Missing policy/language<br>• Lower concentration (30% vs 80%) |

**Synthesis:** 
- Graph analysis identifies *what* patterns exist (hubs, concentration, power-law)
- ABM explains *why* they emerge (economic pull + network effects)
- **Complementary approaches:** Graph reveals patterns → ABM explains mechanisms

In [ ]:
# Visualization: Graph metrics vs ABM predictions
# Compare weighted degree (graph) to final population (ABM)

# Calculate weighted degree from graph
weighted_in_degree = {}
for node in G.nodes():
    total_in = sum(G[u][node]['weight'] for u in G.predecessors(node))
    weighted_in_degree[node] = total_in

# ABM final populations from NB6 (top 10 example - replace with actual if available)
abm_final_pop = {
    'QAT': 922, 'USA': 866, 'AUS': 849, 'GBR': 770, 'ARE': 689,
    'HKG': 665, 'DEU': 640, 'CAN': 639, 'JPN': 571, 'FRA': 409
    # Add more countries if available
}

# Match countries present in both
common_countries = set(weighted_in_degree.keys()) & set(abm_final_pop.keys())

graph_values = [weighted_in_degree[c] for c in common_countries]
abm_values = [abm_final_pop[c] for c in common_countries]

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(graph_values, abm_values, s=100, alpha=0.6, edgecolors='black')

for c in common_countries:
    plt.annotate(c, (weighted_in_degree[c], abm_final_pop[c]), 
                fontsize=9, alpha=0.7)

plt.xlabel('Weighted In-Degree (Graph Analysis)', fontweight='bold', fontsize=12)
plt.ylabel('Final Population (ABM Simulation)', fontweight='bold', fontsize=12)
plt.title('Graph Centrality vs ABM Population Dynamics', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Countries compared: {len(common_countries)}")
print("Graph centrality and ABM population show similar destination importance")

## Section 3: Model Evolution Visualization

**Show the progression from random baseline to calibrated model**

In [ ]:
# Model progression visualization (enhanced from NB6)

models = ['Baseline\n(Random)', 'Economic\nOnly', 'Economic\n+ Network', 
          'Full Model\n(Initial)', 'Full Model\n(Calibrated)']
correlations = [-0.043, -0.1477, 0.0742, 0.1458, 0.489]
colors = ['red', 'darkred', 'orange', 'yellowgreen', 'green']

fig, ax = plt.subplots(figsize=(12, 6))

# Bar chart
bars = ax.bar(range(len(models)), correlations, color=colors, alpha=0.7, 
              edgecolor='black', linewidth=2)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, correlations)):
    y_pos = val + 0.02 if val > 0 else val - 0.03
    ax.text(i, y_pos, f'{val:.3f}', ha='center', fontweight='bold', fontsize=11)

# Add benchmark lines
ax.axhline(0.3, color='green', linestyle='--', linewidth=1.5, alpha=0.5, 
          label='ABM Standard (r=0.3)')
ax.axhline(0.0, color='gray', linestyle='-', linewidth=1, alpha=0.5)

# Annotations
ax.annotate('Mechanism\nconflict!', xy=(1, -0.1477), xytext=(1, -0.25),
           arrowprops=dict(arrowstyle='->', color='red', lw=2),
           fontsize=10, color='red', fontweight='bold')

ax.annotate('Network\nrescue!', xy=(2, 0.0742), xytext=(2, 0.2),
           arrowprops=dict(arrowstyle='->', color='orange', lw=2),
           fontsize=10, color='orange', fontweight='bold')

ax.annotate('Calibration\noptimizes!', xy=(4, 0.489), xytext=(4, 0.35),
           arrowprops=dict(arrowstyle='->', color='green', lw=2),
           fontsize=10, color='green', fontweight='bold')

ax.set_xticks(range(len(models)))
ax.set_xticklabels(models, fontsize=11)
ax.set_ylabel('Pearson Correlation (r)', fontweight='bold', fontsize=12)
ax.set_title('Model Evolution: From Random to Calibrated', fontweight='bold', fontsize=15)
ax.set_ylim(-0.3, 0.55)
ax.legend(fontsize=10, loc='upper left')
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../figures/model_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Model evolution chart saved to ../figures/model_evolution.png")

## Section 4: Mechanism Contribution Breakdown

In [ ]:
# Mechanism weights and contributions

mechanisms = ['Economic\nPull', 'Network\nEffects', 'Distance\nCosts']
weights = [0.6, 0.4, 0.1]  # From calibration
incremental_gains = [None, 0.222, 0.084]  # Network rescued economic, distance refined

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Weight distribution
colors_mech = ['steelblue', 'coral', 'lightgreen']
wedges, texts, autotexts = ax1.pie(weights, labels=mechanisms, autopct='%1.0f%%',
                                    colors=colors_mech, startangle=90,
                                    textprops={'fontsize': 12, 'fontweight': 'bold'})
ax1.set_title('Optimal Mechanism Weights\n(From Calibration)', 
             fontweight='bold', fontsize=13)

# Right: Incremental contributions
contributions = [0.222, 0.084, 0.332]  # Network, Distance, Calibration
labels_contrib = ['Network\nRescue', 'Distance\nRefinement', 'Calibration\nBoost']
colors_contrib = ['coral', 'lightgreen', 'gold']

bars2 = ax2.bar(range(3), contributions, color=colors_contrib, alpha=0.7,
               edgecolor='black', linewidth=2)

for i, (bar, val) in enumerate(zip(bars2, contributions)):
    ax2.text(i, val + 0.01, f'+{val:.3f}', ha='center', fontweight='bold', fontsize=11)

ax2.set_xticks(range(3))
ax2.set_xticklabels(labels_contrib, fontsize=11)
ax2.set_ylabel('Correlation Improvement (Δr)', fontweight='bold', fontsize=12)
ax2.set_title('Incremental Mechanism Contributions', fontweight='bold', fontsize=13)
ax2.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../figures/mechanism_contributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Mechanism breakdown saved to ../figures/mechanism_contributions.png")

## Section 5: (Optional) Simple Scenario Testing

**Note for Seb:** You can implement 1-2 simple scenarios here if interested:
- Modify attractiveness scores (economic shock)
- Remove/add edges (policy changes)
- Use your Agent/World classes from Partner_Work/CITS4403_Group_Project.ipynb

**Skip if time is limited** - NB6 validation is sufficient for the project.

### Scenario Template (Optional)

**Example: USA Economic Crisis**

```python
# Load calibrated parameters from NB6
params_calibrated = {
    'min_wanderlust': 0.1,
    'max_wanderlust': 0.5,
    'min_economic_need': 0.3,
    'max_economic_need': 0.8,
    'economic_weight': 0.6,
    'network_weight': 0.4,
    'distance_weight': 0.1
}

# Modify USA attractiveness (50% drop)
attractiveness_crisis = attractiveness.copy()
attractiveness_crisis['USA'] *= 0.5

# Run simulation
world_crisis = EnhancedWorld(G, attractiveness_crisis, params=params_calibrated)
world_crisis.add_migrants(10000, agent_class=FullMigrant)
world_crisis.run_simulation(n_steps=100)

# Compare baseline vs crisis
# [Visualization code here]
```

In [ ]:
# Optional: Implement scenario here
# Seb - use this space if you want to test a simple scenario

## Section 6: Final Summary

**Project Achievements:**

1. ✅ **Graph Analysis (NB1-4):** Identified empirical patterns
   - Extreme concentration (top 20 = 80%)
   - Power-law distribution (α = -1.81)
   - Hub-spoke topology
   - Major corridors quantified

2. ✅ **Agent-Based Model (NB5-6):** Explained mechanisms
   - Built incremental models (Random → Economic → Network → Full)
   - Calibrated optimal parameters (60/40/10)
   - Achieved r = 0.489 (excellent for migration ABMs)
   - Identified synergy: Network effects rescue economic-only model

3. ✅ **Synthesis (NB7):** Connected approaches
   - Graph reveals *what* patterns exist
   - ABM explains *why* they emerge
   - Complementary insights

**Key Finding:**
> Migration patterns emerge from three interacting mechanisms: economic attractiveness (60%), diaspora networks (40%), and geographic/cultural distance. The synergy between economic pull and network effects is essential—economics alone fails, but combined with chain migration through diaspora communities, the model successfully reproduces real-world patterns.

**Next Step:** Prepare final report document (Word) with detailed methodology, results, and conclusions.

---

## Notes for Seb (Partner Collaboration):

**What you can contribute here:**

1. **Section 2:** Add your graph analysis insights from notebooks 01-03 in Partner_Work
   - Compare your findings to NB1-4 results
   - Add any additional visualizations

2. **Section 5:** Implement 1-2 scenarios using your Agent/World code
   - Your `CITS4403_Group_Project.ipynb` has Agent and Simulation classes
   - Adapt them to use calibrated parameters from NB6
   - Test simple scenarios (economic shock, policy change)

3. **Visualizations:** Enhance any section with better graphics
   - Use your preferred plotting style
   - Add interactive elements if desired (but static is fine)

**Time estimate:** 2-4 hours if adding scenarios, 30 min if just reviewing/polishing

**Priority:** LOW - NB6 is comprehensive. Only add if you want to enhance presentation.